# Ethereum and ERC20 derivation and transaction manual

In [14]:
from web3 import Web3, middleware
from web3.gas_strategies.time_based import fast_gas_price_strategy

from crypto_eth import HDPrivateKey, HDKey

import binascii

import pandas as pd

## Derivation

In [15]:
master_key = HDPrivateKey.master_key_from_mnemonic(
    'trade icon company use feature fee order double inhale gift news long')
root_keys = HDKey.from_path(master_key, "m/44'/60'/0'")
acct_priv_key = root_keys[-1]
private_keys = []

# derivation example for Ethereum generate first 10 addresses
for i in range(10):
    keys = HDKey.from_path(acct_priv_key, '{change}/{index}'.format(change=0, index=i))
    private_key = keys[-1]
    public_key = private_key.public_key
    private_keys.append(private_key)
    print(f'Index {i}:')
    print(f'\tPrivate key: {private_key._key.to_hex()}')
    print(f'\tAddress: {private_key.public_key.address()}')
    print('-' * 100)

Index 0:
	Private key: 396f7f85659d2b43ef05f7abc80586aa3989e8ef7218433350ddd5945f7f9008
	Address: 0x88a007ec4f1819f24c0988fc9c26496b99b436d1
----------------------------------------------------------------------------------------------------
Index 1:
	Private key: 7f31b8e44f7cdf90959ac38f412698640ed4113414e4c0bf074cb3aee9cec72a
	Address: 0xab33d517b6a63a0b1c099b8438d6641cf1a984cc
----------------------------------------------------------------------------------------------------
Index 2:
	Private key: 49edccc09e07e63daf689fd808886837f18cfb87900f5957afea0f633d73c2f8
	Address: 0xb76d3a5d568f339411a6f0697694cddb6396df58
----------------------------------------------------------------------------------------------------
Index 3:
	Private key: 015e90cb0d2f1bebfee8248c1991e13478aaf68279bbd8d7eee839c55f8499b5
	Address: 0xcb64cd644d24b57a96287f61dffeb402b01e23c6
----------------------------------------------------------------------------------------------------
Index 4:
	Private key: 4752f5feb

## Ethereum transction
### Setup Web3

In [26]:
# use infura endpoint
# endpoint_url = 'wss://ropsten.infura.io/ws/adc06f70568e46d88376a8a3a30e5497'
endpoint_url = 'https://ropsten.infura.io/v3/adc06f70568e46d88376a8a3a30e5497'

# Ropsten chainId is 3
chainId = 3

# setup connection
w3 = Web3(Web3.HTTPProvider(endpoint_url))
if not w3.isConnected():
    print('Can\'t connect to Web3 endpoint')

# select gas price strategy
w3.eth.setGasPriceStrategy(fast_gas_price_strategy)

### Get last block

In [27]:
block = w3.eth.getBlock('latest')
print(f'Current block: {block}')

Current block: AttributeDict({'difficulty': 3507008849, 'extraData': HexBytes('0xd983010906846765746889676f312e31312e3133856c696e7578'), 'gasLimit': 8000000, 'gasUsed': 7298193, 'hash': HexBytes('0xa656b780e91c58ae36659e4c0779f95d9fdf4a52535ce284f7351d22724a7fd1'), 'logsBloom': HexBytes('0x0420100400088010000004000400440000000100040408800040000400040000012094000024800000048020800004000001000002030104000000040c3404000000200004002200808000888002004610028000000400808000081108200040088000000a02000080000000004008002100004050001900020000100800200c0000000000000020040030400050a000200104800024040040200000000409000200010410001000200001000010008080000000108082000020000004005240a0009002000000504000040200002c0888306200a100002060020002400060800010802000000086408084000000008000000088108401000882001000000000'), 'miner': '0x4CCfB3039b78d3938588157564c9AD559bAfAB94', 'mixHash': HexBytes('0xe2ba98ad9d17786257604fdc14fb1cb65282f5e0c3f6524ae12f63f681df3765'), 'nonce': HexBytes('0xd7438a2000795908'), 'numbe

### Prepare accounts

In [18]:
# take first two accounts as from and to
from_address = w3.toChecksumAddress(private_keys[0].public_key.address())
from_private_key = private_keys[0]._key.to_hex()

to_address = w3.toChecksumAddress(private_keys[1].public_key.address())
to_private_key = private_keys[1]._key.to_hex()

### Send Ethereum

In [19]:
# fetching nonce (count of transactions)
nonce = w3.eth.getTransactionCount(from_address)

transaction = {
    'to': to_address,
    'value': 200_000, # 0.0000000000002 Ether
    'gas': 21_000,
    'gasPrice': w3.eth.generateGasPrice(),
    'nonce': nonce,
    'chainId': chainId
}

signed = w3.eth.account.sign_transaction(transaction, from_private_key)

print(f'Signed transaction: {binascii.b2a_hex(signed.rawTransaction)}')

transaction_hash = w3.eth.sendRawTransaction(signed.rawTransaction)

print(f'Transaction hash: {binascii.b2a_hex(transaction_hash)}')

Signed transaction: b'f8688203f8845ed9d10182520894ab33d517b6a63a0b1c099b8438d6641cf1a984cc83030d40802aa0a0e5167b0d6cc9b13fbec04286a0dd787e5ec461d2e70f4a8179c2efb25a4bdca03ef72e52a736c2cdbd669952cc41af4a1e5f261689fb81efde3f617d99996a19'
Transaction hash: b'17b6c7618e7ecdb33595999d146356ce62c79582ccc79c9785788b7b005f889a'


### Ethereum balance

In [25]:
addresses = [w3.toChecksumAddress(key.public_key.address()) for key in private_keys]
balances = [w3.fromWei(w3.eth.getBalance(address), 'ether') for address in addresses]

df = pd.DataFrame(zip(addresses,balances), columns = ['Address', 'Balance']) 

df

,Address,Balance
0,0x88a007eC4F1819F24C0988fc9C26496b99b436D1,2.508182778065264123
1,0xab33D517b6A63A0B1C099b8438D6641cF1a984cC,1.002326564527903776
2,0xb76d3a5D568f339411a6f0697694cDDb6396df58,2.154467975998
3,0xcb64cD644D24b57a96287F61DfFeb402B01E23c6,0.998700340002
4,0x8396738b4Cb2A2a7Ca636161346a7cB22Ee06E25,3.979769
5,0xBd666C0Ec86628475b1602aBA0eBF45570A33d4d,3.01
6,0x46FF31E07D54a28c468c300325AfcdAc4c6b7F2e,4
7,0x44874A7d255dF37f5b9C91d173bEE6ab4441c881,4.1
8,0x9a2C78cad9e6c0Dc46820C37704065bb22E5Eab0,3
9,0x577C31aAd54366E7F8Cf931A804F409d20B6928C,2.99948678


### Send ERC20

In [21]:
# read abi for ERC20 contract
with open('erc20_abi.json', 'r') as file:
    erc20_abi = file.read()

In [23]:
# Another Test Coin (ATC) ERC20
erc20_contract_address = "0x96f55B7d683EB46FF23aBEf2aFb41EB54FCF524b"
erc20_contract = w3.eth.contract(address=erc20_contract_address, abi=erc20_abi)

# fetching nonce (count of transactions)
nonce = w3.eth.getTransactionCount(from_address)

erc20_transaction = erc20_contract.functions.transfer(erc20_contract_address, 1).buildTransaction(
    {
        'chainId': chainId,
        'gas': 100_000,
        'gasPrice': w3.eth.generateGasPrice(),
        'nonce': nonce,
    })

print(erc20_transaction)

signed_erc20_transaction = w3.eth.account.sign_transaction(erc20_transaction, from_private_key)

print(f'Signed transaction: {binascii.b2a_hex(signed_erc20_transaction.rawTransaction)}')

erc20_transaction_hash = w3.eth.sendRawTransaction(signed_erc20_transaction.rawTransaction)

print(f'Transaction hash: {binascii.b2a_hex(erc20_transaction_hash)}')

{'value': 0, 'chainId': 3, 'gas': 100000, 'gasPrice': 1545285073, 'nonce': 1017, 'to': '0x96f55B7d683EB46FF23aBEf2aFb41EB54FCF524b', 'data': '0xa9059cbb00000000000000000000000096f55b7d683eb46ff23abef2afb41eb54fcf524b0000000000000000000000000000000000000000000000000000000000000001'}
Signed transaction: b'f8ab8203f9845c1b2dd1830186a09496f55b7d683eb46ff23abef2afb41eb54fcf524b80b844a9059cbb00000000000000000000000096f55b7d683eb46ff23abef2afb41eb54fcf524b00000000000000000000000000000000000000000000000000000000000000012aa04cdbe1e6f2f0ef3d1cac8a457b92d50bbc0fa941407ef194318c7f4db432eeffa0731bd5201f1d1f400f592a8789d8a35d2e7e8faea81900d8e7e2c09712cf9aa4'
Transaction hash: b'8a2e8fc7f79763a7a5730e007b371f35d9ae304d86e4cb95297a99966f2a8513'


### ERC20 balances

In [28]:
decimals = 10 ** erc20_contract.functions.decimals().call()

erc20_addresses = [w3.toChecksumAddress(key.public_key.address()) for key in private_keys]
erc20_balances = [(erc20_contract.functions.balanceOf(address).call() / decimals) for address in erc20_addresses]

df = pd.DataFrame(zip(erc20_addresses, erc20_balances), columns=['Address', 'Balance'])

df

,Address,Balance
0,0x88a007eC4F1819F24C0988fc9C26496b99b436D1,6689.273300
1,0xab33D517b6A63A0B1C099b8438D6641cF1a984cC,179.315696
2,0xb76d3a5D568f339411a6f0697694cDDb6396df58,208.992688
3,0xcb64cD644D24b57a96287F61DfFeb402B01E23c6,112.454000
4,0x8396738b4Cb2A2a7Ca636161346a7cB22Ee06E25,109.750000
5,0xBd666C0Ec86628475b1602aBA0eBF45570A33d4d,52.740000
6,0x46FF31E07D54a28c468c300325AfcdAc4c6b7F2e,131.410000
7,0x44874A7d255dF37f5b9C91d173bEE6ab4441c881,105.320000
8,0x9a2C78cad9e6c0Dc46820C37704065bb22E5Eab0,108.510000
9,0x577C31aAd54366E7F8Cf931A804F409d20B6928C,33.590000
